In [1]:
import fastText

In [2]:
fasttext = fastText.load_model('fasttext.bin')


In [3]:
import pandas as pd
import numpy as np
from progressbar import progressbar as pb

df = pd.read_csv('all_data_fixed3.csv')

In [4]:
print(len(df))
df.head()

5330


,0,1,2,3,4,5
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1
4,Для выхода из экономического кризиса вызванног...,ввело 8 часовой рабочий день в промышленности,ликвидировало помещичье землевладение,отменило выкупные платежи,ввело продовольственную разверстку,4


In [5]:
lst_df = df.values.tolist()
lst_df[0]

['Совещательный орган при императоре в начале XX в',
 'Сенат',
 'Государственный совет',
 'Консилия министров',
 'Верховный тайный совет',
 2]

In [6]:
def get_vec(lst):
    res = np.zeros((5,300))
    for i in range(len(lst)-1):
        cur_vec = np.zeros(300)
        cur_tag = lst[i].split(' ')
        for c in cur_tag:
            try:
                cur_vec += fasttext.get_word_vector(c)
            except:
                continue
        res[i,:] = cur_vec
    return res

In [7]:
from multiprocessing import Pool

p = Pool(6)

FT_embs = p.map(get_vec, lst_df)
FT_embs = np.array(FT_embs)

In [8]:
import pickle

with open('fasttext_embed3.pickle', 'wb') as f:
    pickle.dump(FT_embs, f)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity as cos
cosines = np.zeros((FT_embs.shape[0], FT_embs.shape[1]-1))
cosines.shape

(5330, 4)

In [10]:
for i in range(len(df)):
    for j in range(1,5):
        cosines[i, j-1] = cos(FT_embs[i,0].reshape(1,-1), FT_embs[i,j].reshape(1,-1))[0]

In [11]:
cosines

array([[ 0.19440531,  0.17442491, -0.01326479,  0.19964368],
       [ 0.18635438,  0.16732294,  0.26197196,  0.24486485],
       [ 0.20581724,  0.33038647,  0.20055023,  0.17308868],
       ...,
       [ 0.25240314,  0.34019593,  0.35046837,  0.34019593],
       [ 0.27919196,  0.15199645,  0.17402604,  0.21909671],
       [ 0.13518369,  0.186899  ,  0.12405629,  0.12276438]])

In [12]:
preds = np.argmax(cosines, axis=1)
preds = preds + 1
preds

array([4, 3, 2, ..., 3, 1, 2])

In [13]:
target = np.array(df['5'], int)
target

array([2, 2, 3, ..., 2, 1, 1])

In [14]:
from sklearn.metrics import accuracy_score as acc

acc(target, preds)

0.326078799249531

## LEMMATIZED

In [15]:
from pymystem3 import Mystem
import nltk
import re
mystem = Mystem() 

def nlp_process(text):
    tokens = nltk.word_tokenize(text)
    clear_tokens = []
    for token in tokens:
        new_token = re.sub(r'[^\w\s]', '', token)
        if new_token != '':
            new_token = mystem.lemmatize(token)
            clear_tokens.append(new_token[0])
    return ' '.join(clear_tokens)

In [16]:
df_stemmed = df.copy()
names = ['0', '1', '2', '3', '4']

for name in names:
    df_stemmed[name] = [nlp_process(text) for text in df[name]]
df_stemmed

,0,1,2,3,4,5
0,совещательный орган при император в начало XX в,сенат,государственный совет,консилий министр,верховный тайный совет,2
1,министр внутренний дело с 1904 г либерал автор...,лорис меликов,святополк мирский,милюков,витта,2
2,какой из называть событие происходить 27 январ...,подрыв флагманский корабль петропавловск,высадка японский войско на ляодунский полуостров,героический бой варяг и кореец,заключение портсмутский мирный договор,3
3,что из называть относиться к результат первый ...,появление многопартийность,формирование конституционный монархия,ликвидация сословный строй,введение рабочий контроль за производство,1
4,для выход из экономический кризис вызывать пер...,вводить 8 часовой рабочий день в промышленность,ликвидировать помещичий землевладение,отменять выкупной платеж,вводить продовольственный разверстка,4
5,творчество поэт гумилев ахматова мандельштам в...,акмеизм,футуризм,импрессионизм,реализм,1
6,самый многочисленный сословие российский империя,мещанин,казачество,крестьянство,разночинец,3
7,усиление позиция россия в китай вызывать в пер...,сша,япония,великобритания,германия,2
8,какой из называть событие происходить рано другой,восстание на броненосец князь потемкин тавриче...,восстание на крейсер очаков,стачка текстильщик в иваново вознесенск,созыв первый государственный дума,3
9,отмечать о ком идти речь дворянин политический...,столыпин,милюков,родзянко,львов,3


In [17]:
lst_df_lemm = df_stemmed.values.tolist()

In [18]:
from multiprocessing import Pool

p = Pool(6)

FT_embs_lemm = p.map(get_vec, lst_df_lemm)
FT_embs_lemm = np.array(FT_embs_lemm)

In [19]:
import pickle

with open('fasttext_embed_lemm3.pickle', 'wb') as f:
    pickle.dump(FT_embs_lemm, f)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity as cos
cosines = np.zeros((FT_embs_lemm.shape[0], FT_embs_lemm.shape[1]-1))
cosines.shape

(5330, 4)

In [21]:
for i in range(len(df)):
    for j in range(1,5):
        cosines[i, j-1] = cos(FT_embs_lemm[i,0].reshape(1,-1), FT_embs_lemm[i,j].reshape(1,-1))[0]

In [22]:
cosines

array([[ 0.2049735 ,  0.19350092,  0.06790652,  0.21344542],
       [ 0.03459496,  0.17356271, -0.00959812,  0.09914419],
       [ 0.2023272 ,  0.35063089,  0.26119753,  0.13590529],
       ...,
       [ 0.31017261,  0.32570848,  0.35385786,  0.32570848],
       [ 0.37400792,  0.24187348,  0.25267692,  0.27424472],
       [ 0.18337317,  0.14563994,  0.18253171,  0.17495403]])

In [23]:
preds = np.argmax(cosines, axis=1)
preds = preds + 1
preds

array([4, 2, 2, ..., 3, 1, 1])

In [24]:
target = np.array(df['5'], int)
target

array([2, 2, 3, ..., 2, 1, 1])

In [25]:
from sklearn.metrics import accuracy_score as acc

acc(target, preds)

0.2885553470919325